In [3]:
# !pip install kaggle
# !pip install boto3
# !pip install pandas
# !pip install numpy

# !export PATH = "/content/.local/bin/kaggle"
# !export PATH = "/content/.local/lib/python3.6/site-packages/kaggle-1.3.12.dist-info/*"
# !export PATH = "/content/.local/lib/python3.6/site-packages/kaggle/*"
# !ln -s ~/.local/bin/kaggle /usr/bin/kaggle
# !wget https://github.com/fatenaught/home_risk/blob/master/kaggle.json
# !cp kaggle.json .kaggle 
# !chmod 755 .kaggle/kaggle.json
# !kaggle competitions download -c home-credit-default-risk

    100% |████████████████████████████████| 737kB 7.0MB/s 


In [1]:
import psutil
memory = psutil.virtual_memory().free / (1024.0 ** 3)
print(memory)
print(psutil.virtual_memory())

11.600753784179688
svmem(total=13655240704, available=12976599040, percent=5.0, used=385642496, free=12456079360, active=464302080, inactive=495648768, buffers=163147776, cached=650371072, shared=811008, slab=174313472)


In [2]:
!pip install lightgbm
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from google.colab import files
import xgboost as xgb
import lightgbm as lgb
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import Imputer
pd.set_option('display.max_columns', 500)
import random

# Functions

In [0]:
def accuracy_check(score,cm):
  plt.figure(figsize=(9,9))
  sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r');
  plt.ylabel('Actual label');
  plt.xlabel('Predicted label');
  all_sample_title = 'Accuracy Score: {0}'.format(score)
  plt.title(all_sample_title, size = 15);

# Import Data

In [0]:
application_train_original = pd.read_csv("https://s3.amazonaws.com/home-risk/application_train.csv")
application_test = pd.read_csv("https://s3.amazonaws.com/home-risk/application_test.csv")
#bureau = pd.read_csv("https://s3.amazonaws.com/home-risk/bureau.csv")
#bureau_balance = pd.read_csv("https://s3.amazonaws.com/home-risk/bureau_balance.csv")
# credit_card_balance = pd.read_csv("https://s3.amazonaws.com/home-risk/credit_card_balance.csv")
# install_payments = pd.read_csv("https://s3.amazonaws.com/home-risk/installments_payments.csv")
previous_applications = pd.read_csv("https://s3.amazonaws.com/home-risk/previous_application.csv")
# pos_cash = pd.read_csv("https://s3.amazonaws.com/home-risk/POS_CASH_balance.csv")

In [4]:
print(application_train_original.shape)
application_train_original.head(3)

(307511, 122)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [0]:
application_train_original['number_of_null'] = application_train_original.isnull().sum(axis = 1).values
application_train = pd.get_dummies(application_train_original)
application_train = application_train.fillna(10000000)


# Main Table Modeling
## Main Table Featrue Engineering

In [0]:
mis_val_percent = pd.DataFrame(100*application_train_original.isnull().sum(axis = 1)/\
                               len(list(application_train_original.columns)))
mis_val_percent.columns = ['% of Total Columns']
mis_val_percent = pd.DataFrame(mis_val_percent[mis_val_percent.iloc[:,0] != 0].\
                               sort_values('% of Total Columns', ascending=False).round(1))
mis_val_percent.columns = ['percentage_null_value']

In [42]:
for i in [5, 15, 25, 35, 45]:
  threshold = i
  cross_table = \
   pd.crosstab(mis_val_percent['percentage_null_value'] > threshold, application_train_original['TARGET'])

  print('threshold >', str(threshold),
        'difference',(cross_table.iloc[1,1]/(cross_table.iloc[1,1]+cross_table.iloc[1,0]) - \
        cross_table.iloc[0,1]/(cross_table.iloc[0,1] + cross_table.iloc[0,0])).round(3))
cross_table

threshold > 5 difference 0.019
threshold > 15 difference 0.019
threshold > 25 difference 0.022
threshold > 35 difference 0.022
threshold > 45 difference 0.043


TARGET,0,1
percentage_null_value,,
False,264279,22861
True,10331,1438


## Anomaly

In [0]:
application_train.loc[application_train['AMT_INCOME_TOTAL'] == 100000000, 'AMT_INCOME_TOTAL'] = np.nan

application_train['DAYS_BIRTH'] = np.abs(application_train['DAYS_BIRTH'])

application_train['DAYS_EMPLOYED'] = np.abs(application_train['DAYS_EMPLOYED'])
application_train.loc[application_train['DAYS_EMPLOYED'] == 365243, 'DAYS_EMPLOYED'] = np.nan

application_train['DAYS_REGISTRATION'] = np.abs(application_train['DAYS_REGISTRATION'])

application_train['DAYS_ID_PUBLISH'] = np.abs(application_train['DAYS_ID_PUBLISH'])

application_train.loc[application_train['REGION_RATING_CLIENT_W_CITY'] == -1,
                      'REGION_RATING_CLIENT_W_CITY'] = np.nan

application_train.loc[application_train['OBS_30_CNT_SOCIAL_CIRCLE'] > 100, 
                      'OBS_30_CNT_SOCIAL_CIRCLE'] = np.nan
application_train.loc[application_train['OBS_60_CNT_SOCIAL_CIRCLE'] > 100, 
                      'OBS_60_CNT_SOCIAL_CIRCLE'] = np.nan

application_train['DAYS_LAST_PHONE_CHANGE'] = np.abs(application_train['DAYS_LAST_PHONE_CHANGE'])
application_train.loc[application_train['AMT_REQ_CREDIT_BUREAU_QRT'] > 100, 
                      'AMT_REQ_CREDIT_BUREAU_QRT'] = np.nan


In [7]:
Counter(application_train.TARGET)

Counter({0: 282686, 1: 24825})

## Poly

In [6]:
# pick the original feature for additional engineering
poly_features = application_train[['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH','TARGET']]
poly_features_test = application_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]

# imputer for handling missing values
imputer = Imputer(strategy = 'median')
poly_target = poly_features['TARGET']
poly_features = poly_features.drop(columns = ['TARGET'])

# Need to impute missing values
poly_features = imputer.fit_transform(poly_features)
poly_features_test = imputer.transform(poly_features_test)
                                  
# Create the polynomial object with specified degree
poly_transformer = PolynomialFeatures(degree = 3)

# Train the polynomial features
poly_transformer.fit(poly_features)

PolynomialFeatures(degree=3, include_bias=True, interaction_only=False)

In [7]:
# Transform the features
data_poly_features = poly_transformer.transform(poly_features)
data_poly_features_test = poly_transformer.transform(poly_features_test)
print('Polynomial Features shape: ', poly_features.shape)

# poly_transformer.get_feature_names(input_features = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 
#                                                      'EXT_SOURCE_3', 'DAYS_BIRTH'])[:15]

Polynomial Features shape:  (307511, 4)


In [0]:
data_poly_features = \
      pd.DataFrame(data_poly_features, columns = \
                   poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 
                                                       'EXT_SOURCE_3', 'DAYS_BIRTH']))

data_poly_features_test = \
      pd.DataFrame(data_poly_features_test, columns = \
                   poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 
                                                       'EXT_SOURCE_3', 'DAYS_BIRTH']))

# add back target
data_poly_features['TARGET'] = poly_target
# Find the correlations with the target
poly_corrs = data_poly_features.corr()['TARGET'].sort_values()

In [0]:
# Merge polynomial features into training dataframe
data_poly_features['SK_ID_CURR'] = application_train['SK_ID_CURR']
data_poly_features = data_poly_features.drop('TARGET',axis = 1)
data_poly_features = \
  data_poly_features.drop(['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH', '1'],axis = 1)
application_train = application_train.merge(data_poly_features, on = 'SK_ID_CURR', how = 'left')

# Merge polnomial features into testing dataframe
data_poly_features_test['SK_ID_CURR'] = application_test['SK_ID_CURR']
data_poly_features_test = \
 data_poly_features_test.drop(['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH', '1'],axis = 1)
application_test = application_test.merge(data_poly_features_test, on = 'SK_ID_CURR', how = 'left')

In [0]:
del data_poly_features_test
del data_poly_features

In [59]:
application_test.head(3)

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,EXT_SOURCE_1^2,EXT_SOURCE_1 EXT_SOURCE_2,EXT_SOURCE_1 EXT_SOURCE_3,EXT_SOURCE_1 DAYS_BIRTH,EXT_SOURCE_2^2,EXT_SOURCE_2 EXT_SOURCE_3,EXT_SOURCE_2 DAYS_BIRTH,EXT_SOURCE_3^2,EXT_SOURCE_3 DAYS_BIRTH,DAYS_BIRTH^2,EXT_SOURCE_1^3,EXT_SOURCE_1^2 EXT_SOURCE_2,EXT_SOURCE_1^2 EXT_SOURCE_3,EXT_SOURCE_1^2 DAYS_BIRTH,EXT_SOURCE_1 EXT_SOURCE_2^2,EXT_SOURCE_1 EXT_SOURCE_2 EXT_SOURCE_3,EXT_SOURCE_1 EXT_SOURCE_2 DAYS_BIRTH,EXT_SOURCE_1 EXT_SOURCE_3^2,EXT_SOURCE_1 EXT_SOURCE_3 DAYS_BIRTH,EXT_SOURCE_1 DAYS_BIRTH^2,EXT_SOURCE_2^3,EXT_SOURCE_2^2 EXT_SOURCE_3,EXT_SOURCE_2^2 DAYS_BIRTH,EXT_SOURCE_2 EXT_SOURCE_3^2,EXT_SOURCE_2 EXT_SOURCE_3 DAYS_BIRTH,EXT_SOURCE_2 DAYS_BIRTH^2,EXT_SOURCE_3^3,EXT_SOURCE_3^2 DAYS_BIRTH,EXT_SOURCE_3 DAYS_BIRTH^2,DAYS_BIRTH^3
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,Unaccompanied,Working,Higher education,Married,House / apartment,0.018850,-19241,-2329,-5170.0,-812,NaN,1,1,0,1,0,1,NaN,2.0,2,2,TUESDAY,18,0,0,0,0,0,0,Kindergarten,0.752614,0.789654,0.159520,0.066,0.059,0.9732,NaN,NaN,NaN,0.1379,0.125,NaN,NaN,NaN,0.0505,NaN,NaN,0.0672,0.0612,0.9732,NaN,NaN,NaN,0.1379,0.125,NaN,NaN,NaN,0.0526,NaN,NaN,0.0666,0.059,0.9732,NaN,NaN,NaN,0.1379,0.125,NaN,NaN,NaN,0.0514,NaN,NaN,NaN,block of flats,0.0392,"Stone, brick",No,0.0,0.0,0.0,0.0,-1740.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,5.664286e-01,5.943053e-01,1.200567e-01,-1.448106e+04,0.623554,0.125965,-15193.739370,0.025446,-3069.315478,370216081.0,4.263024e-01,4.472828e-01,9.035643e-02,-1.089865e+04,4.692958e-01,9.480331e-02,-1.143503e+04,1.915139e-02,-2.310011e+03,2.786300e+08,0.492392,0.099469,-11997.802403,0.020094,-2423.698322,2.923427e+08,0.004059,-489.615795,5.905670e+07,-7.123328e+12
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,Unaccompanied,Working,Secondary / 

##Linear Regression

In [0]:
application_train = application_train.fillna(application_train.mean()['SK_ID_CURR':'AMT_REQ_CREDIT_BUREAU_YEAR'])
application_data = application_train.drop(columns=['SK_ID_CURR', 'TARGET'])
application_target = application_train['TARGET']
x_train, x_test, y_train, y_test = train_test_split(application_data, application_target, test_size= 0.25, random_state = 0)

In [0]:
model_logit = LogisticRegression(class_weight = 'balanced')
model_logit.fit(x_train,y_train)
logit_score_train = metrics.roc_auc_score(y_train, model_logit.predict(x_train))
logit_score_test = metrics.roc_auc_score(y_test, model_logit.predict(x_test))

logit_score_test = model_logit.score(x_test,y_test) 
logit_cm_test = metrics.confusion_matrix(y_test, model_logit.predict(x_test))
accuracy_check(logit_score, logit_cm)
print(logit_score_test)

## Random Forest


**Model V1**


</b> Feature - Main Table</br>
</b> Optimal Tuning</br>

* </b> max_depth: 10 n_tree: 20 max_feature: None train_roc: 0.816 val_roc: 0.733 </br>
* </b>  max_depth: 10 n_tree: 70 max_feature: 48 train_roc: 0.816 val_roc: 0.741</b> 







In [0]:
application_data = application_train.drop(columns=['SK_ID_CURR', 'TARGET'])
application_target = application_train['TARGET']
x_train_ori, x_test, y_train_ori, y_test = \
     train_test_split(application_data, application_target, test_size = 0.1, random_state = 1)
x_train, x_val, y_train, y_val = \
     train_test_split(x_train_ori, y_train_ori, test_size = 0.25, random_state = 1)

In [17]:
model_rf_version = []
para_max_depth = []
para_n_tree = []
para_max_feature = []
metric_roc_val = []
metric_roc_train = []

i = 1
for value_n_tree in [90, 120, 150]: 
  for value_max_depth in [10]:
    for value_max_feature in [50]:
      
      model_rf = \
        RandomForestClassifier(class_weight = 'balanced', verbose = 0, n_jobs = -1, 
                               n_estimators = value_n_tree, max_depth = value_max_depth, 
                               max_features = value_max_feature)
      
      model_rf.fit(x_train, y_train)
      
      roc_val = \
         metrics.roc_auc_score(y_val, [j[1] for j in model_rf.predict_proba(x_val)]).round(3)
      roc_train = \
         metrics.roc_auc_score(y_train, [j[1] for j in model_rf.predict_proba(x_train)]).round(3)
      
      model_rf_version.append(model_rf)
      para_max_depth.append(value_max_depth)
      para_n_tree.append(value_n_tree)
      para_max_feature.append(str(value_max_feature))
      metric_roc_val.append(roc_val)
      metric_roc_train.append(roc_train)
      
      print('iter', i, '=>', 'max_depth:', value_max_depth, 'n_tree:', value_n_tree, 
            'max_feature:', value_max_feature, 'train_roc:', roc_train, 'val_roc:', roc_val)
      i += 1           
      

iter 1 => max_depth: 10 n_tree: 90 max_feature: 50 train_roc: 0.815 val_roc: 0.74
iter 2 => max_depth: 10 n_tree: 120 max_feature: 50 train_roc: 0.818 val_roc: 0.741
iter 3 => max_depth: 10 n_tree: 150 max_feature: 50 train_roc: 0.817 val_roc: 0.741


In [0]:
model_rf_tuning = pd.DataFrame(para_max_depth, columns=['max_depth'])
model_rf_tuning['n_tree'] = para_n_tree
model_rf_tuning['max_feature'] = para_max_feature
model_rf_tuning['n_tree'] = para_n_tree
model_rf_tuning['roc_val'] = metric_roc_val
model_rf_tuning['roc_train'] = metric_roc_train
model_rf_tuning.to_csv('model_rf_tuning_indepth.csv', index = False, index_label = False)
files.download('model_rf_tuning_indepth.csv')

In [0]:
feature_importance = pd.DataFrame(x_train.columns, columns=['feature_name'])
feature_importance['importance'] = model_rf.feature_importances_.round(2)
feature_importance = feature_importance.sort_values(by = 'importance', ascending = False)
feature_importance.head(10)

In [20]:
metrics.roc_auc_score(y_test, [i[1].round(3) for i in model_rf_version[np.argsort(metric_roc_val)[-1]].predict_proba(x_test)]).round(3)

0.75

## Gradient Boost Method

In [0]:
lgb_train = lgb.Dataset(data = x_train, label = y_train)
lgb_val = lgb.Dataset(data = x_val , label = y_val)

params = {
    'task': 'train',
    'device' : 'cpu',
    'nthread': 20,
    'tree_learner' : 'voting',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'num_leaves': 100,
    'metric': 'auc',
    'learning_rate': 0.1,
    'max_depth' : 20,
    'verbose': 0
}

gbm = lgb.train(
    params,
    lgb_train,
    num_boost_round = 5000,
    valid_sets = lgb_val,
    early_stopping_rounds = 200,
    verbose_eval = 25
)

In [31]:
metrics.roc_auc_score(y_test, gbm.predict(x_test))

0.7105122006951905

# Merge Bureau, Bureau Balance Table

## Bureau Balance

In [7]:
print(bureau_balance.head())

   SK_ID_BUREAU  MONTHS_BALANCE STATUS
0       5715448               0      C
1       5715448               1      C
2       5715448               2      C
3       5715448               3      C
4       5715448               4      C


In [0]:
bureau_balance.columns
bureau_balance['MONTHS_BALANCE'] = np.abs(bureau_balance['MONTHS_BALANCE'])

#Bureau_Balance Status - Mean Method
bureau_balance_copy = bureau_balance.copy()
bureau_balance_copy = pd.get_dummies(bureau_balance_copy)
bureau_balance_mean = pd.DataFrame(bureau_balance_copy.groupby(['SK_ID_BUREAU']).mean().reset_index(drop = False))
bureau_balance_max = pd.DataFrame(bureau_balance_copy.groupby(['SK_ID_BUREAU']).max().reset_index(drop = False))
bureau_balance_min = pd.DataFrame(bureau_balance_copy.groupby(['SK_ID_BUREAU']).min().reset_index(drop = False))
bureau_balance_median = pd.DataFrame(bureau_balance_copy.groupby(['SK_ID_BUREAU']).median().reset_index(drop = False))
bureau_balance_train = bureau_balance_mean.merge(bureau_balance_max, on = 'SK_ID_BUREAU')\
                       .merge(bureau_balance_min, on = 'SK_ID_BUREAU')\
                       .merge(bureau_balance_median, on = 'SK_ID_BUREAU')
bureau_balance_train.head(5)


In [0]:
#Count Method 
#bureau_balance_month_mean = pd.DataFrame(bureau_balance.groupby(['SK_ID_BUREAU']).mean())
#bureau_balance_month_mean = bureau_balance_month_mean.reset_index(drop = False)
# bureau_balance_status = \
#   pd.DataFrame(bureau_balance.groupby(['SK_ID_BUREAU','STATUS']).size().unstack())
# bureau_balance_status.columns = ['Bureau_DPD_' + j for j in ['0','1','2','3','4','5','C','X']]
# bureau_balance_status = bureau_balance_status.reset_index(drop = False)
# bureau_balance_train = bureau_balance_month_mean.merge(bureau_balance_status, on = 'SK_ID_BUREAU')

##Bureau  = Bureau + Bureau Balance

In [0]:
bureau_train = bureau.merge(bureau_balance_train, on = 'SK_ID_BUREAU')

#feature engineering
#0
bureau_train['DAYS_CREDIT'] = np.abs(bureau_train['DAYS_CREDIT'])
#1,2
bureau_train = bureau_train.merge(\
                                 bureau_train.groupby(['SK_ID_CURR'])['DAYS_CREDIT'].count().reset_index().rename(index=str, columns={'DAYS_CREDIT': 'BUREAU_LOAN_COUNT'}),\
                                 on = 'SK_ID_CURR')\
                           .merge(\
                                 bureau_train.groupby(['SK_ID_CURR'])['CREDIT_TYPE'].nunique().reset_index().rename(index=str, columns={'CREDIT_TYPE': 'BUREAU_LOAN_TYPES'}),\
                                 on = 'SK_ID_CURR')
#3
bureau_train['BUREAU_LOAN_AVERAGE_TYPECOUNT'] = bureau_train['BUREAU_LOAN_COUNT'] / bureau_train['BUREAU_LOAN_TYPES']

#4
CREDIT_ACTIVE_COUNT_TEMP = pd.DataFrame(bureau_train.groupby(['SK_ID_CURR','CREDIT_ACTIVE'])['DAYS_CREDIT'].count().reset_index().\
                           rename(index=str,columns = {'DAYS_CREDIT':'CREDIT_ACTIVE_COUNT'}))

bureau_train = bureau_train.merge(\
                                 CREDIT_ACTIVE_COUNT_TEMP[(CREDIT_ACTIVE_COUNT_TEMP.CREDIT_ACTIVE == 'Active')].drop(['CREDIT_ACTIVE'],axis = 1),\
                                 on = 'SK_ID_CURR')
bureau_train['CREDIT_ACTIVE_PERCENT'] = bureau_train['CREDIT_ACTIVE_COUNT'] / bureau_train['BUREAU_LOAN_COUNT']
bureau_train = bureau_train.drop(['CREDIT_ACTIVE_COUNT'],axis = 1)

#5
AVERAGE_DIFFERENCE_DAYS_TEMP = bureau_train[['SK_ID_CURR', 'SK_ID_BUREAU', 'DAYS_CREDIT']].groupby(['SK_ID_CURR'])
AVERAGE_DIFFERENCE_DAYS_TEMP = AVERAGE_DIFFERENCE_DAYS_TEMP.apply(lambda x: x.sort_values(['DAYS_CREDIT'], ascending = True)).reset_index(drop = True)
AVERAGE_DIFFERENCE_DAYS_TEMP['DAYS_DIFF'] = AVERAGE_DIFFERENCE_DAYS_TEMP.groupby(['SK_ID_CURR'])['DAYS_CREDIT'].diff()
AVERAGE_DIFFERENCE_DAYS_TEMP['DAYS_DIFF'] = AVERAGE_DIFFERENCE_DAYS_TEMP['DAYS_DIFF'].fillna(0).astype('uint32')
AVERAGE_DIFFERENCE_DAYS_TEMP = AVERAGE_DIFFERENCE_DAYS_TEMP.drop(['SK_ID_CURR','DAYS_CREDIT'],axis = 1)
bureau_train = bureau_train.merge(AVERAGE_DIFFERENCE_DAYS_TEMP[['SK_ID_BUREAU','DAYS_DIFF']],on = 'SK_ID_BUREAU')

gc.collect()
print(bureau_train.head())



In [0]:
#Bureau - Mean Method
bureau_train = bureau.merge(bureau_balance_train, on = 'SK_ID_BUREAU')
bureau_train['DAYS_CREDIT'] = np.abs(bureau_train['DAYS_CREDIT'])

bureau_train = pd.get_dummies(bureau_train)
bureau_train_mean = pd.DataFrame(bureau_train.groupby(['SK_ID_CURR']).mean().reset_index(drop = False))
bureau_train_max = pd.DataFrame(bureau_train.groupby(['SK_ID_CURR']).max().reset_index(drop = False))
bureau_train_min = pd.DataFrame(bureau_train.groupby(['SK_ID_CURR']).min().reset_index(drop = False))
bureau_train_median = pd.DataFrame(bureau_train.groupby(['SK_ID_CURR']).median().reset_index(drop = False))

bureau_train_final = bureau_train_mean.merge(bureau_train_max,on = 'SK_ID_CURR')\
                     .merge(bureau_train_min, on = 'SK_ID_CURR')\
                     .merge(bureau_train_median, on = 'SK_ID_CURR')
bureau_train_final = bureau_train_final.reset_index(drop = False)
bureau_train_final.fillna(10000000)

bureau_train_final.head()

#application_bureau_data = application_train.merge(bureau_train_final, on = 'SK_ID_CURR', how = 'left')
#application_bureau_data = application_bureau_data.fillna(10000000)
#application_bureau_data.head(5)

In [0]:
# Count Method
# bureau_train = bureau.merge(bureau_balance_train, on = 'SK_ID_BUREAU')
# bureau_train['DAYS_CREDIT'] = np.abs(bureau_train['DAYS_CREDIT'])


# bureau_mean = pd.DataFrame(bureau_train.groupby(['SK_ID_CURR']).mean())
# bureau_mean = bureau_mean.reset_index(drop = False)

# bureau_sum = pd.DataFrame(bureau_train.groupby(['SK_ID_CURR']).sum())
# bureau_sum = bureau_sum.reset_index(drop = False)


# bureau_train = bureau_train.drop(['SK_ID_BUREAU'], axis=1)
# bureau_mean = bureau_mean.drop(['SK_ID_BUREAU'], axis=1)
# bureau_sum = bureau_sum.drop(['SK_ID_BUREAU'], axis=1)

# bureau_mean.columns = \
# [bureau_mean.columns[0]] + [str('mean_') + i for i in bureau_mean.columns[1::]]

# bureau_sum.columns = \
# [bureau_sum.columns[0]] + [str('sum_') + i for i in bureau_sum.columns[1::]]


# # CREDIT_ACTIVE, CREDIT_CURRENCY, CREDIT_TYPE
# bureau_CA = pd.DataFrame(bureau_bureau_balance.\
#                          groupby(['SK_ID_CURR','CREDIT_ACTIVE']).size().unstack())
# bureau_CA = bureau_CA.reset_index(drop = False)

# bureau_CC = pd.DataFrame(bureau_bureau_balance.\
#                          groupby(['SK_ID_CURR','CREDIT_CURRENCY']).size().unstack())
# bureau_CC = bureau_CC.reset_index(drop = False)

# bureau_CT = pd.DataFrame(bureau_bureau_balance.\
#                          groupby(['SK_ID_CURR','CREDIT_TYPE']).size().unstack())
# bureau_CT = bureau_CT.reset_index(drop = False)

# bureau_train_final = bureau_mean\
#                                 .merge(bureau_sum, on = 'SK_ID_CURR', how = 'left')\
#                                 .merge(bureau_CA, on = 'SK_ID_CURR', how = 'left')\
#                                 .merge(bureau_CC, on = 'SK_ID_CURR', how = 'left')\
#                                 .merge(bureau_CT, on = 'SK_ID_CURR', how = 'left')

## Bureau RF Model

In [0]:
x_train_ori, x_test, y_train_ori, y_test  = \
    train_test_split(application_bureau_data.drop(['SK_ID_CURR', 'TARGET'], axis = 1), 
                     application_bureau_data['TARGET'], 
                     test_size = 0.1, random_state = 1)

x_train, x_val, y_train, y_val = \
    train_test_split(x_train_ori, y_train_ori, test_size = 0.25, random_state = 1)

In [15]:
model_rf_version = []
para_max_depth = []
para_n_tree = []
para_max_feature = []
metric_roc_val = []
metric_roc_train = []

i = 1
for value_n_tree in [70]: 
  for value_max_depth in [10]:
    for value_max_feature in [48]:
      
      model_rf_v2 = \
        RandomForestClassifier(class_weight = 'balanced', verbose = 0, n_jobs = -1, 
                               n_estimators = value_n_tree, max_depth = value_max_depth, 
                               max_features = value_max_feature)
      
      model_rf_v2.fit(x_train, y_train)
      
      roc_val = \
         metrics.roc_auc_score(y_val, [j[1] for j in model_rf_v2.predict_proba(x_val)]).round(3)
      roc_train = \
         metrics.roc_auc_score(y_train, [j[1] for j in model_rf_v2.predict_proba(x_train)]).round(3)
      
      model_rf_version.append(model_rf_v2)
      para_max_depth.append(value_max_depth)
      para_n_tree.append(value_n_tree)
      para_max_feature.append(str(value_max_feature))
      metric_roc_val.append(roc_val)
      metric_roc_train.append(roc_train)
      
      print('iter', i, '=>', 'max_depth:', value_max_depth, 'n_tree:', value_n_tree, 
            'max_feature:', value_max_feature, 'train_roc:', roc_train, 'val_roc:', roc_val)
      i += 1           
      

iter 1 => max_depth: 10 n_tree: 70 max_feature: 48 train_roc: 0.82 val_roc: 0.733


In [16]:
metrics.roc_auc_score(y_test, [j[1] for j in model_rf_v2.predict_proba(x_test)]).round(3)

0.74

In [116]:
feature_importance = pd.DataFrame(x_train.columns, columns=['feature_name'])
feature_importance['importance'] = model_rf_v2.feature_importances_.round(2)
feature_importance = feature_importance.sort_values(by = 'importance', ascending = False)
feature_importance.head(20)

,feature_name,importance
28,EXT_SOURCE_2,0.20
29,EXT_SOURCE_3,0.19
27,EXT_SOURCE_1,0.05
7,DAYS_EMPLOYED,0.04
6,DAYS_BIRTH,0.04
9,DAYS_ID_PUBLISH,0.02
129,NAME_EDUCATION_TYPE_Higher education,0.02
77,DAYS_LAST_PHONE_CHANGE,0.02
4,AMT_GOODS_PRICE,0.02
3,AMT_ANNUITY_x,0.02


In [73]:
print(list(feature_importance['feature_name']))

['EXT_SOURCE_2', 'EXT_SOURCE_3', 'EXT_SOURCE_1', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DAYS_ID_PUBLISH', 'DAYS_LAST_PHONE_CHANGE', 'NAME_EDUCATION_TYPE_Higher education', 'AMT_INCOME_TOTAL', 'HOUR_APPR_PROCESS_START', 'AMT_REQ_CREDIT_BUREAU_YEAR', 'AMT_REQ_CREDIT_BUREAU_QRT', 'CODE_GENDER_F', 'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'CODE_GENDER_M', 'NAME_EDUCATION_TYPE_Secondary / secondary special', 'OWN_CAR_AGE', 'DAYS_REGISTRATION', 'REGION_POPULATION_RELATIVE', 'ORGANIZATION_TYPE_Security Ministries', 'ORGANIZATION_TYPE_Services', 'ORGANIZATION_TYPE_Telecom', 'ORGANIZATION_TYPE_Self-employed', 'CNT_CHILDREN', 'ORGANIZATION_TYPE_Security', 'ORGANIZATION_TYPE_School', 'ORGANIZATION_TYPE_Trade: type 2', 'ORGANIZATION_TYPE_Restaurant', 'ORGANIZATION_TYPE_Religion', 'ORGANIZATION_TYPE_Realtor', 'ORGANIZATION_TYPE_Postal', 'ORGANIZATION_TYPE_Trade: type 1', 'ORGANIZATION_TYPE_Transport: type 2', 'O

# Test for Submit

## Application Data Only

In [37]:
#align train and test features
y_train_align = application_train_original['TARGET']

x_train = pd.get_dummies(application_train_original)
x_train = x_train.fillna(10000000)
x_test_test = pd.get_dummies(application_test)
x_test_test = x_test_test.fillna(10000000)

x_train, x_test_test = x_train.align(x_test_test, join = 'inner', axis = 1)

print('Training Features shape: ', x_train.shape)
print('Testing Features shape: ', x_test_test.shape)

y_train = y_train_align

x_train = x_train.drop(['SK_ID_CURR'],axis = 1)
print(len(x_train.columns))

def submit(model,model_ver):
  submit = submit = application_test[['SK_ID_CURR']]
  submit['TARGET'] = [i[1] for i in model.predict_proba(x_test_test)]
  submit.head()
  file_name = 'submit_' + str(model_ver) + '.csv'
  submit.to_csv(file_name, index = False, index_label = False)
  files.download(file_name)
  

Training Features shape:  (307511, 242)
Testing Features shape:  (48744, 242)
241


### RF

In [0]:
model_rf_v3 = RandomForestClassifier(class_weight = 'balanced', verbose = 0, n_jobs = -1, 
                               n_estimators = 70, max_depth = 10, max_features = 48)
model_rf_v3.fit(x_train,y_train)


x_test_test = x_test_test.drop(['SK_ID_CURR'], axis = 1)
submit(model_rf_v3,1)



# Feature Engineering Previous Application History

In [11]:
previous_applications.head(3)

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0


In [0]:
previous_applications_process = previous_applications.copy()
previous_applications_process['NAME_TYPE_SUITE'] = \
            previous_applications_process['NAME_TYPE_SUITE'].fillna('Unknown')
previous_applications_process['PRODUCT_COMBINATION'] = \
            previous_applications_process['PRODUCT_COMBINATION'].fillna('Unknown')


In [0]:
previous_applications_dummy = pd.get_dummies(previous_applications_process)
previous_applications_dummy = previous_applications_dummy.drop(['SK_ID_PREV'], axis=1)

In [0]:
previous_applications_dummy_max = previous_applications_dummy.groupby(['SK_ID_CURR']).max()
previous_applications_dummy_max.columns = ['max_' + i for i in previous_applications_dummy_max.columns] 
previous_applications_dummy_max.reset_index(inplace = True)

In [0]:
# previous_applications_dummy_min = previous_applications_dummy.groupby(['SK_ID_CURR']).min()
# previous_applications_dummy_min.columns = ['min_' + i for i in previous_applications_dummy_min.columns] 
# previous_applications_dummy_min.reset_index(inplace = True)


In [0]:
previous_applications_dummy_mean = previous_applications_dummy.groupby(['SK_ID_CURR']).mean()
previous_applications_dummy_mean.columns = ['mean_' + i for i in previous_applications_dummy_mean.columns] 
previous_applications_dummy_mean.reset_index(inplace = True)

In [0]:
previous_applications_dummy_median = previous_applications_dummy.groupby(['SK_ID_CURR']).median()
previous_applications_dummy_median.columns = \
               ['median_' + i for i in previous_applications_dummy_median.columns] 
previous_applications_dummy_median.reset_index(inplace = True)

In [0]:
previous_applications_dummy_sum = previous_applications_dummy.groupby(['SK_ID_CURR']).sum()
previous_applications_dummy_sum.columns = ['sum_' + i for i in previous_applications_dummy_sum.columns] 
previous_applications_dummy_sum.reset_index(inplace = True)

In [0]:
# previous_applications_dummy_cnt = previous_applications_dummy.groupby(['SK_ID_CURR']).size()
# previous_applications_dummy_cnt = pd.DataFrame(previous_applications_dummy_cnt, columns = ['pre_count'])
# previous_applications_dummy_cnt.columns = ['cnt_' + i for i in previous_applications_dummy_cnt.columns]
# previous_applications_dummy_cnt.reset_index(inplace = True)

In [0]:
application_pre_train = \
     application_train.merge(previous_applications_dummy_max, on = ['SK_ID_CURR'], how = 'left')\
                      .merge(previous_applications_dummy_mean, on = ['SK_ID_CURR'], how = 'left')\
                      .merge(previous_applications_dummy_sum, on = ['SK_ID_CURR'], how = 'left')\
                      .merge(previous_applications_dummy_median, on = ['SK_ID_CURR'], how = 'left')
#.merge(bureau_train_final, on = ['SK_ID_CURR'], how = 'left')
#.merge(previous_applications_dummy_max, on = ['SK_ID_CURR'], how = 'left') # worked
#.merge(previous_applications_dummy_mean, on = ['SK_ID_CURR'], how = 'left') # worked
#.merge(previous_applications_dummy_min, on = ['SK_ID_CURR'], how = 'left') # not work
#.merge(previous_applications_dummy_sum, on = ['SK_ID_CURR'], how = 'left') # not work
#.merge(previous_applications_dummy_cnt, on = ['SK_ID_CURR'], how = 'left') # not work

application_pre_train = application_pre_train.fillna(10000000)

In [0]:
del previous_applications_dummy
del previous_applications_process
# del previous_applications_dummy_max
# del previous_applications_dummy_mean
#del previous_applications_dummy_median
#del previous_applications_dummy_sum
#del previous_applications_dummy_min

In [0]:
x_train_ori, x_test, y_train_ori, y_test  = \
    train_test_split(application_pre_train.drop(['SK_ID_CURR', 'TARGET'], axis = 1), 
                     application_pre_train['TARGET'], 
                     test_size = 0.2, random_state = 1)

In [0]:
# del application_train_original
# del bureau
# del bureau_balance

In [22]:
i = 1
for value_n_tree in [200]: 
  for value_max_depth in [15]:
    for value_max_feature in [80]:
      
      model_rf_v4 = \
        RandomForestClassifier(class_weight = 'balanced', verbose = 0, n_jobs = -1, 
                               n_estimators = value_n_tree, max_depth = value_max_depth, 
                               max_features = value_max_feature)
      
      model_rf_v4.fit(x_train_ori, y_train_ori)
      
      roc_val = \
         metrics.roc_auc_score(y_test, 
                               [j[1] for j in model_rf_v4.predict_proba(x_test)]).round(3)
      roc_train = \
         metrics.roc_auc_score(y_train_ori, 
                               [j[1] for j in model_rf_v4.predict_proba(x_train_ori)]).round(3)
      
      
      print('iter', i, '=>', 'max_depth:', value_max_depth, 'n_tree:', value_n_tree, 
            'max_feature:', value_max_feature, 'train_roc:', roc_train, 'val_roc:', roc_val)
      i += 1           

iter 1 => max_depth: 15 n_tree: 200 max_feature: 80 train_roc: 0.975 val_roc: 0.746


In [15]:
Counter(y_train_ori)

Counter({0: 226153, 1: 19855})

In [21]:
model_store = []
for i in range(8):
  print(i)
  index_negative = x_train_ori[y_train_ori == 0].index
  index_negative_sample = np.random.choice(index_negative, size = 19855, replace = False)

  y_train_balanced = list(y_train_ori[y_train_ori == 1]) +  list(y_train_ori[index_negative_sample])
  x_train_balanced = pd.concat([x_train_ori[y_train_ori == 1], 
                                x_train_ori[x_train_ori.index.isin(index_negative_sample)]], axis=0)

  lgb_train = lgb.Dataset(data = x_train_balanced, label = y_train_balanced)
  lgb_val = lgb.Dataset(data = x_test , label = y_test)
  params = {
      'task': 'train',
      'device' : 'cpu',
      'tree_learner' : 'voting',
      'boosting_type': 'gbdt',
      'objective': 'binary',
      'num_leaves': 80,
      'metric': 'auc',
      'learning_rate': 0.05,
      #'max_depth' : 30,
      'verbose': 0
  }
  gbm_v2 = lgb.train(
      params,
      lgb_train,
      num_boost_round = 5000,
      valid_sets = lgb_val,
      early_stopping_rounds = 200,
      verbose_eval = 25
     )
  model_store.append(gbm_v2)

0
Training until validation scores don't improve for 200 rounds.
[25]	valid_0's auc: 0.755917
[50]	valid_0's auc: 0.766153
[75]	valid_0's auc: 0.771747
[100]	valid_0's auc: 0.774174
[125]	valid_0's auc: 0.775187
[150]	valid_0's auc: 0.775422
[175]	valid_0's auc: 0.77519
[200]	valid_0's auc: 0.775193
[225]	valid_0's auc: 0.774965
[250]	valid_0's auc: 0.774882
[275]	valid_0's auc: 0.774716
[300]	valid_0's auc: 0.774654
[325]	valid_0's auc: 0.774563
Early stopping, best iteration is:
[139]	valid_0's auc: 0.775481
1
Training until validation scores don't improve for 200 rounds.
[25]	valid_0's auc: 0.755998
[50]	valid_0's auc: 0.766141
[75]	valid_0's auc: 0.771581
[100]	valid_0's auc: 0.774211
[125]	valid_0's auc: 0.774958
[150]	valid_0's auc: 0.775415
[175]	valid_0's auc: 0.775869
[200]	valid_0's auc: 0.776078
[225]	valid_0's auc: 0.776083
[250]	valid_0's auc: 0.775942
[275]	valid_0's auc: 0.775813
[300]	valid_0's auc: 0.775561
[325]	valid_0's auc: 0.775181
[350]	valid_0's auc: 0.774661
[3

In [0]:
data_feature_importance = pd.DataFrame(gbm_v2.feature_name(), columns=['feature_name'])
data_feature_importance['importance'] = gbm_v2.feature_importance()
data_feature_importance.sort_values(by = ['importance'], inplace=True, ascending = False)

In [23]:
prediction_test_combine = pd.DataFrame(y_test.values, columns=['y_test'])
for i, m in enumerate(model_store):
  print(i)
  prediction_test_combine['v' + str(i)] = np.round(m.predict(x_test),3)

0
1
2
3
4
5
6
7


In [25]:
x_train_combine, x_test_combine, y_train_combine, y_test_combine  = \
    train_test_split(prediction_test_combine.drop(['y_test'], axis = 1), 
                     prediction_test_combine['y_test'], 
                     test_size = 0.2, random_state = 1)

lgb_train_cb = lgb.Dataset(data = x_train_combine, label = y_train_combine)
lgb_val_cb = lgb.Dataset(data = x_test_combine , label = y_test_combine)

params = {
      'task': 'train',
      'device' : 'cpu',
      'tree_learner' : 'voting',
      'boosting_type': 'gbdt',
      'objective': 'binary',
      'num_leaves': 80,
      'metric': 'auc',
      'learning_rate': 0.05,
      'verbose': 0
  }

gbm_v3 = lgb.train(
      params,
      lgb_train_cb,
      num_boost_round = 5000,
      valid_sets = lgb_val_cb,
      early_stopping_rounds = 200,
      verbose_eval = 25
     )

Training until validation scores don't improve for 200 rounds.
[25]	valid_0's auc: 0.775056
[50]	valid_0's auc: 0.775337
[75]	valid_0's auc: 0.77346
[100]	valid_0's auc: 0.773433
[125]	valid_0's auc: 0.772439
[150]	valid_0's auc: 0.771569
[175]	valid_0's auc: 0.769773
[200]	valid_0's auc: 0.768065
Early stopping, best iteration is:
[8]	valid_0's auc: 0.777018


In [26]:
metrics.roc_auc_score(y_test, 
                      prediction_test_combine.drop(['y_test'], axis=1).mean(axis = 1).values).round(3)

0.782

In [37]:
metrics.roc_auc_score(y_test_combine, gbm_v3.predict(x_test_combine)).round(3)

0.777

In [28]:
data_feature_importance.head(20)

,feature_name,importance
2,AMT_CREDIT,297
3,AMT_ANNUITY,289
7,DAYS_EMPLOYED,244
9,DAYS_ID_PUBLISH,232
6,DAYS_BIRTH,214
8,DAYS_REGISTRATION,188
77,DAYS_LAST_PHONE_CHANGE,182
4,AMT_GOODS_PRICE,172
28,EXT_SOURCE_2,170
268,EXT_SOURCE_2_EXT_SOURCE_3^2,164


In [29]:
application_test_data = pd.get_dummies(application_test)
application_test_data = application_test_data.fillna(10000000)
application_test_data.loc[application_test_data['AMT_INCOME_TOTAL'] == 100000000, 
                          'AMT_INCOME_TOTAL'] = np.nan
application_test_data['DAYS_BIRTH'] = np.abs(application_test_data['DAYS_BIRTH'])
application_test_data['DAYS_EMPLOYED'] = np.abs(application_test_data['DAYS_EMPLOYED'])
application_test_data.loc[application_test_data['DAYS_EMPLOYED'] == 365243, 'DAYS_EMPLOYED'] = np.nan
application_test_data['DAYS_REGISTRATION'] = np.abs(application_test_data['DAYS_REGISTRATION'])
application_test_data['DAYS_ID_PUBLISH'] = np.abs(application_test_data['DAYS_ID_PUBLISH'])
application_test_data.loc[application_test_data['REGION_RATING_CLIENT_W_CITY'] == -1,
                      'REGION_RATING_CLIENT_W_CITY'] = np.nan
application_test_data.loc[application_test_data['OBS_30_CNT_SOCIAL_CIRCLE'] > 100, 
                      'OBS_30_CNT_SOCIAL_CIRCLE'] = np.nan
application_test_data.loc[application_test_data['OBS_60_CNT_SOCIAL_CIRCLE'] > 100, 
                      'OBS_60_CNT_SOCIAL_CIRCLE'] = np.nan
application_test_data['DAYS_LAST_PHONE_CHANGE'] = np.abs(application_test_data['DAYS_LAST_PHONE_CHANGE'])
application_test_data.loc[application_test_data['AMT_REQ_CREDIT_BUREAU_QRT'] > 100, 
                      'AMT_REQ_CREDIT_BUREAU_QRT'] = np.nan

application_test_data = application_test_data.loc[:, application_train.columns]
application_test_data = \
     application_test_data.merge(previous_applications_dummy_max, on = ['SK_ID_CURR'], how = 'left')\
                          .merge(previous_applications_dummy_mean, on = ['SK_ID_CURR'], how = 'left')\
                          .merge(previous_applications_dummy_sum, on = ['SK_ID_CURR'], how = 'left')\
                          .merge(previous_applications_dummy_median, on = ['SK_ID_CURR'], how = 'left')
application_test_data = application_test_data.loc[:,x_train_ori.columns]
application_test_data = application_test_data.fillna(10000000)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [30]:
prediction = pd.DataFrame(application_test['SK_ID_CURR'].values, columns = ['SK_ID_CURR'])
for i, m in enumerate(model_store):
  print(i)
  prediction['v' + str(i)] = np.round(m.predict(application_test_data),3)

0
1
2
3
4
5
6
7


In [0]:
prediction_final = prediction.drop(['SK_ID_CURR'], axis=1).mean(axis=1)

In [0]:
submission = pd.DataFrame(application_test['SK_ID_CURR'].values, columns = ['SK_ID_CURR'])
submission['TARGET'] = gbm_v3.predict(application_test_data)
#submisstion['TARGET'] = prediction_final
#submisstion['TARGET'] = np.round(gbm_v2.predict(application_test_data),3)

In [42]:
print(submission['TARGET'].nunique())
submission.head(10)

2


,SK_ID_CURR,TARGET
0,100001,0.174561
1,100005,0.174561
2,100013,0.174561
3,100028,0.223987
4,100038,0.223987
5,100042,0.174561
6,100057,0.223987
7,100065,0.174561
8,100066,0.174561
9,100067,0.223987


In [0]:
submission.to_csv('submission_v5.csv', index = False, index_label = False)
files.download('submission_v5.csv')